In [ ]:
!pip install ipython-sql prettytable

import prettytable

prettytable.DEFAULT = 'DEFAULT'

In [79]:
import csv
import pandas as pd
import sqlite3

conn=sqlite3.connect('FinalDB.db')


In [112]:
import pandas as pd

url1 = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv'
url2 = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv'
url3 = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv'
data1 = pd.read_csv(url1)
data2 = pd.read_csv(url2)
data3 = pd.read_csv(url3)
data1.head()
data2.head()
data3.head()

,ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,...,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
0,3512276,HK587712,2004-08-28,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,False,False,...,9,14.0,58.0,6,1155838.0,1873050.0,2004,41.807440,-87.703956,"(41.8074405, -87.703955849)"
1,3406613,HK456306,2004-06-26,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,False,False,...,11,27.0,23.0,6,1152206.0,1906127.0,2004,41.898280,-87.716406,"(41.898279962, -87.716405505)"
2,8002131,HT233595,2011-04-04,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,False,False,...,2,3.0,38.0,6,1177436.0,1876313.0,2011,41.815933,-87.624642,"(41.815933131, -87.624642127)"
3,7903289,HT133522,2010-12-30,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,...,4,7.0,46.0,6,1194622.0,1850125.0,2010,41.743665,-87.562463,"(41.743665322, -87.562462756)"
4,10402076,HZ138551,2016-02-02,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,False,False,...,8,15.0,66.0,6,1155240.0,1860661.0,2016,41.773455,-87.706480,"(41.773455295, -87.706480471)"


In [113]:
con=sqlite3.connect('FinalDB.db')
data1.to_sql('CHICAGO_PUBLIC_SCHOOLS', con, if_exists='replace', index=False)
data2.to_sql('CENSUS_DATA', con, if_exists='replace', index=False)
data3.to_sql('CHICAGO_CRIME_DATA', con, if_exists='replace', index=False)

533

In [115]:
query = "SELECT COUNT(*) AS total_crimes FROM CHICAGO_CRIME_DATA;"
result = pd.read_sql_query(query, con)
print(result)


   total_crimes
0           533


In [120]:
query = """
SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, PER_CAPITA_INCOME 
FROM CENSUS_DATA 
WHERE PER_CAPITA_INCOME < 11000;
"""
result = pd.read_sql_query(query, con)
print(result)


   COMMUNITY_AREA_NUMBER COMMUNITY_AREA_NAME  PER_CAPITA_INCOME
0                   26.0  West Garfield Park              10934
1                   30.0      South Lawndale              10402
2                   37.0         Fuller Park              10432
3                   54.0           Riverdale               8201


In [121]:
query = """
SELECT CASE_NUMBER 
FROM CHICAGO_CRIME_DATA 
WHERE DESCRIPTION LIKE '%MINOR%' AND DESCRIPTION NOT LIKE '%CHILD%';
"""
result = pd.read_sql_query(query, con)
print(result)


  CASE_NUMBER
0    HL266884
1    HK238408


In [122]:
query = """
SELECT * 
FROM CHICAGO_CRIME_DATA 
WHERE PRIMARY_TYPE = 'KIDNAPPING' AND DESCRIPTION LIKE '%CHILD%';
"""
result = pd.read_sql_query(query, con)
print(result)


        ID CASE_NUMBER        DATE                 BLOCK  IUCR PRIMARY_TYPE  \
0  5276766    HN144152  2007-01-26  050XX W VAN BUREN ST  1792   KIDNAPPING   

                DESCRIPTION LOCATION_DESCRIPTION  ARREST  DOMESTIC  ...  \
0  CHILD ABDUCTION/STRANGER               STREET       0         0  ...   

   DISTRICT  WARD  COMMUNITY_AREA_NUMBER  FBICODE X_COORDINATE  Y_COORDINATE  \
0        15  29.0                   25.0       20    1143050.0     1897546.0   

   YEAR   LATITUDE  LONGITUDE                       LOCATION  
0  2007  41.874908 -87.750249  (41.874908413, -87.750249307)  

[1 rows x 21 columns]


In [123]:
query = """
SELECT DISTINCT PRIMARY_TYPE 
FROM CHICAGO_CRIME_DATA 
WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%';
"""
result = pd.read_sql_query(query, con)
print(result)


             PRIMARY_TYPE
0                 BATTERY
1         CRIMINAL DAMAGE
2               NARCOTICS
3                 ASSAULT
4       CRIMINAL TRESPASS
5  PUBLIC PEACE VIOLATION


In [124]:
query = """
SELECT 
    "Elementary, Middle, or High School" AS School_Type, 
    AVG(SAFETY_SCORE) AS Average_Safety_Score
FROM 
    CHICAGO_PUBLIC_SCHOOLS
GROUP BY 
    "Elementary, Middle, or High School";
"""
result = pd.read_sql_query(query, con)
print(result)


  School_Type  Average_Safety_Score
0          ES             49.520384
1          HS             49.623529
2          MS             48.000000


In [125]:
query = """
SELECT COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, 
       "% HOUSEHOLDS BELOW POVERTY" AS POVERTY_PERCENT
FROM CENSUS_DATA
ORDER BY "% HOUSEHOLDS BELOW POVERTY" DESC
LIMIT 5;
"""
result = pd.read_sql_query(query, con)
print(result)


   COMMUNITY_AREA_NUMBER COMMUNITY_AREA_NAME             POVERTY_PERCENT
0                    1.0         Rogers Park  % HOUSEHOLDS BELOW POVERTY
1                    2.0          West Ridge  % HOUSEHOLDS BELOW POVERTY
2                    3.0              Uptown  % HOUSEHOLDS BELOW POVERTY
3                    4.0      Lincoln Square  % HOUSEHOLDS BELOW POVERTY
4                    5.0        North Center  % HOUSEHOLDS BELOW POVERTY


In [128]:
query = """
SELECT COMMUNITY_AREA_NUMBER 
FROM CHICAGO_CRIME_DATA
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY COUNT(*) DESC
LIMIT 1;
"""
result = pd.read_sql_query(query, con)
print(result)


   COMMUNITY_AREA_NUMBER
0                   25.0


In [131]:
query='''
SELECT COMMUNITY_AREA_NAME 
FROM CENSUS_DATA
WHERE HARDSHIP_INDEX = (
    SELECT MAX(HARDSHIP_INDEX) 
    FROM CENSUS_DATA
);

'''
result=pd.read_sql_query(query,con)
print(result)

  COMMUNITY_AREA_NAME
0           Riverdale


In [132]:
query = """
SELECT COMMUNITY_AREA_NAME 
FROM CENSUS_DATA
WHERE COMMUNITY_AREA_NUMBER = (
    SELECT COMMUNITY_AREA_NUMBER 
    FROM CHICAGO_CRIME_DATA
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY COUNT(*) DESC
    LIMIT 1
);
"""
result = pd.read_sql_query(query, con)
print(result)


  COMMUNITY_AREA_NAME
0              Austin
